In [1]:
!pip install -U gensim --user

In [2]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from gensim import corpora, models
import nltk
import feedparser

In [3]:
class IdentifyingTopicExample:
    def getDocuments(self):
        url = 'https://sports.yahoo.com/mlb/rss.xml'
        feed = feedparser.parse(url)
        self.documents = []
        for entry in feed['entries'][:5]:
            text = entry['summary']
            if 'ex' in text:
                continue
            self.documents.append(text)
            print("-- {}".format(text))
        print("INFO: Fetching documents from {} completed".format(url))

    def cleanDocuments(self):
        tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
        en_stop = set(stopwords.words('english'))
        self.cleaned = []
        for doc in self.documents:
            lowercase_doc = doc.lower()
            words = tokenizer.tokenize(lowercase_doc)
            non_stopped_words = [i for i in words if not i in en_stop]
            self.cleaned.append(non_stopped_words)
        print("INFO: Clearning {} documents completed".format(len(self.documents)))

    def doLDA(self):
        dictionary = corpora.Dictionary(self.cleaned)
        corpus = [dictionary.doc2bow(cleandoc) for cleandoc in self.cleaned]
        ldamodel = models.ldamodel.LdaModel(corpus, num_topics=2, id2word = dictionary)
        print(ldamodel.print_topics(num_topics=2, num_words=4))

    def run(self):
        self.getDocuments()
        self.cleanDocuments()
        self.doLDA()

In [4]:
if __name__ == '__main__':
    topicExample = IdentifyingTopicExample()
    topicExample.run()

-- Houston shrugged off Atlanta's first-inning grand slam in a World Series elimination game, casually fighting back to keep its season alive.
-- It was a rough night for Joe Judge and the New York Giants both on the field and off.
-- Atlanta's Max Fried was a pitcher without a home heading into his senior year of high school, but he overcame the setback on his major league path.
-- After two up-and-down seasons, Atlanta wasn't sure Austin Riley was the answer at third base. Instead of dealing him the 24-year-old became a star.
INFO: Fetching documents from https://sports.yahoo.com/mlb/rss.xml completed
INFO: Clearning 4 documents completed
[(0, '0.033*"atlanta" + 0.026*"home" + 0.025*"year" + 0.025*"school"'), (1, '0.040*"atlanta" + 0.029*"year" + 0.021*"seasons" + 0.021*"sure"')]
